In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [20]:
chorales = open('major_chorales.txt', 'r').read().splitlines() #chorales in the format I want
chorales[:8]

['I V6 I vi I6 IV IV6 I I IV viidim6 I6 viidim/V V vi6 ii V I IV iii6 IV65 viidim I VI64 V7 I V6 I I64 V I I I6 V V6 I I6 V I6 ii vi iii6 ii6 V I vi iii IV IV6 I viidim6 iii vi7 V65/V V I',
 'I I I6 IV vi V6/V V V/V V/V V6 viidim/V V viidim6/V V6 vi6 V64 V/V V V I V6/vi vi V7 V/V I6 IV7 V vi V6 I64 viidim/V V vi V65/V V7 I',
 'I vi V I VII/iii iii vi65 ii V7/V V I ii I6 iii V iii vi64 ii65 V V7 I V V viidim6/ii ii ii ii64 iihdim65/ii iihdim65/ii vi ii V/ii V/ii V6/ii V/ii ii ii ii6 viihdim65 viihdim7 V7/vi vi V/vi vi vi64 IV7 I6 viidim6 viidim I V V I6 IV IV6 iii6 IV65 V65 I ii65 V V7 I',
 'I I V6 I IV6 I6 V7 I IV V IV6 I ii6 V V I I IV V6/V V V42 I6 I6 ii/IV IV I V/V I6 V vi6 V/V V I6 IV6 viihdim42 I64 V I6 V I',
 'I IV viidim6 I V6 IV6 V65 I I I ii7 I6 I6 V iii vi7 viidim6/V V iii vi V I6 I V V6/vi vi V6 I V65/ii ii ii V/ii I V6 V V iii6 I6 V65/V V V42 I6 ii I viidim6 I6 V I',
 'I vi iii IV V42 I6 ii65 V I I V6/vi V7/vi vi V42 I6 ii65 V I I IV viidim64 I viidim6/vi vi V6/vi vi viihdi

In [21]:
chorale_lists = open('all_major_chorales.txt', 'r').read().splitlines() #chorales as lists with each new item a roman numeral
print(len(chorale_lists))

3630


In [22]:
chorale_sets = open('major_chorale_lists.txt', 'r').read().splitlines() #chorales as lists with each new item a roman numeral
chorales[:8]

['I V6 I vi I6 IV IV6 I I IV viidim6 I6 viidim/V V vi6 ii V I IV iii6 IV65 viidim I VI64 V7 I V6 I I64 V I I I6 V V6 I I6 V I6 ii vi iii6 ii6 V I vi iii IV IV6 I viidim6 iii vi7 V65/V V I',
 'I I I6 IV vi V6/V V V/V V/V V6 viidim/V V viidim6/V V6 vi6 V64 V/V V V I V6/vi vi V7 V/V I6 IV7 V vi V6 I64 viidim/V V vi V65/V V7 I',
 'I vi V I VII/iii iii vi65 ii V7/V V I ii I6 iii V iii vi64 ii65 V V7 I V V viidim6/ii ii ii ii64 iihdim65/ii iihdim65/ii vi ii V/ii V/ii V6/ii V/ii ii ii ii6 viihdim65 viihdim7 V7/vi vi V/vi vi vi64 IV7 I6 viidim6 viidim I V V I6 IV IV6 iii6 IV65 V65 I ii65 V V7 I',
 'I I V6 I IV6 I6 V7 I IV V IV6 I ii6 V V I I IV V6/V V V42 I6 I6 ii/IV IV I V/V I6 V vi6 V/V V I6 IV6 viihdim42 I64 V I6 V I',
 'I IV viidim6 I V6 IV6 V65 I I I ii7 I6 I6 V iii vi7 viidim6/V V iii vi V I6 I V V6/vi vi V6 I V65/ii ii ii V/ii I V6 V V iii6 I6 V65/V V V42 I6 ii I viidim6 I6 V I',
 'I vi iii IV V42 I6 ii65 V I I V6/vi V7/vi vi V42 I6 ii65 V I I IV viidim64 I viidim6/vi vi V6/vi vi viihdi

In [149]:
#list of roman numerals
nums = [] 
for i in range(len(chorale_lists)):
    if chorale_lists[i] not in nums:
        nums.append(chorale_lists[i])
nums = sorted(nums)

In [150]:
#tokenization
stoi = {s:i for i,s in enumerate(nums)}
# stoi[','] = 150
# stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{0: '<e>', 1: '<s>', 2: 'I', 3: 'I42', 4: 'I43', 5: 'I43/V', 6: 'I6', 7: 'I6/viidim', 8: 'I64', 9: 'I65', 10: 'I65/V', 11: 'I7', 12: 'III6/viidim', 13: 'IV', 14: 'IV/IV', 15: 'IV/iii', 16: 'IV/vi', 17: 'IV/viidim', 18: 'IV42', 19: 'IV42/iii', 20: 'IV42/vi', 21: 'IV43', 22: 'IV43/iii', 23: 'IV6', 24: 'IV6/IV', 25: 'IV6/vi', 26: 'IV64', 27: 'IV64/ii', 28: 'IV65', 29: 'IV65/iii', 30: 'IV65/vi', 31: 'IV7', 32: 'V', 33: 'V/V', 34: 'V/ii', 35: 'V/vi', 36: 'V42', 37: 'V42/IV', 38: 'V42/V', 39: 'V43', 40: 'V43/V', 41: 'V43/iii', 42: 'V6', 43: 'V6/V', 44: 'V6/ii', 45: 'V6/vi', 46: 'V64', 47: 'V64/V', 48: 'V65', 49: 'V65/IV', 50: 'V65/V', 51: 'V65/ii', 52: 'V65/vi', 53: 'V7', 54: 'V7/IV', 55: 'V7/V', 56: 'V7/ii', 57: 'V7/iii', 58: 'V7/vi', 59: 'VI/ii', 60: 'VI42/viidim', 61: 'VI6/ii', 62: 'VI7/ii', 63: 'VII/iii', 64: 'VII/viidim', 65: 'VII6/iii', 66: 'VII6/viidim', 67: 'VII65/iii', 68: 'VII7/ii', 69: 'VII7/iii', 70: 'Vflat6/iii', 71: 'ii', 72: 'ii/IV', 73: 'ii42', 74: 'ii43', 75: 'ii6', 76: 'ii6

In [151]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ' '.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [152]:
# Train and test splits
# data = []
# train_data = []
# val_data = []

# for i in range(len(chorale_sets)):
#     datai = torch.tensor(encode(chorale_sets[i]), dtype=torch.long)
#     n = int(0.9*len(datai)) # first 90% will be train, rest val
#     train_data.append(datai[:n])
#     val_data.append(datai[n:])

In [165]:
# Train and test splits
data = torch.tensor(encode(chorale_lists), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [166]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [167]:
X, Y = get_batch('train')
X.shape, Y.shape

(torch.Size([32, 4]), torch.Size([32, 4]))

In [168]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [169]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [170]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [171]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [172]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [173]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 4 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 5e-5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [174]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.218773 M parameters


In [175]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 5.2580, val loss 5.2069
step 100: train loss 4.1438, val loss 4.1372
step 200: train loss 3.7692, val loss 3.7848
step 300: train loss 3.5903, val loss 3.5779
step 400: train loss 3.4444, val loss 3.4868
step 500: train loss 3.3494, val loss 3.4011
step 600: train loss 3.2509, val loss 3.3164
step 700: train loss 3.1891, val loss 3.2119
step 800: train loss 3.1276, val loss 3.1721
step 900: train loss 3.0570, val loss 3.1256
step 1000: train loss 3.0191, val loss 3.0701
step 1100: train loss 2.9631, val loss 3.0517
step 1200: train loss 2.9355, val loss 3.0351
step 1300: train loss 2.8925, val loss 3.0212
step 1400: train loss 2.8831, val loss 2.9828
step 1500: train loss 2.8309, val loss 2.9929
step 1600: train loss 2.7941, val loss 2.9590
step 1700: train loss 2.8064, val loss 2.9576
step 1800: train loss 2.7414, val loss 2.9361
step 1900: train loss 2.7092, val loss 2.9332
step 2000: train loss 2.6973, val loss 2.9136
step 2100: train loss 2.6672, val loss 2.9276


In [179]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))

<e> <s> I V ii42 I I IV I IV ii V viidim6 vi V I V I I6 IV vi7 ii I6 V V I6 iii IV6 I V I iidim6/iii viidim6/iii viifdim42/ii V I vi6 viidim6/vi vi vi IV V/vi vi IV vi ii65 V I V/V V I I65 IV viidim64 V6 vi viihdim42 viihdim7 I V6 I IV I6 V7 I IV6 I64 ii65 I6 V65/V V I IV I viidim6 V65 vi65 V/V V I V6/ii ii V6 vi7 viihdim7 I IV V V65/V I6 V V7 I vi I IV V I IV6 V6 I ii6 viidim6/IV V64 vi65 V/V V V iii7 IV IV6 I V IV6 I V V V7 I <e> <s> I vi vi V65/V V V I6 V VII6/iii iii V6 vi65 V/V V V I viidim6/V V V I ii viidim6/V V6 vi65 viidim64 viihdim43/IV viifdim7/iii iii IV42/vi V64 vi65 V64 ii65 V I <e> <s> I I6 V65/V V V V V/V I64 iv7/ii iii6 V7 I IV V I6 ii ii42 I I V6 I IV6 I64 ii65 V I ii64 viihdim7 viidim6/iii v/iii viidim6/V V6 vi65 V/V V I I viidim6/V V V6 vi65 V/V V
